In [1]:
!pip uninstall onnxruntime
!pip install onnxruntime-gpu


^CFound existing installation: onnxruntime 1.22.0
Uninstalling onnxruntime-1.22.0:
  Would remove:
    c:\users\usudl\anaconda3\envs\yolov-env\lib\site-packages\onnxruntime-1.22.0.dist-info\*
    c:\users\usudl\anaconda3\envs\yolov-env\lib\site-packages\onnxruntime\*
    c:\users\usudl\anaconda3\envs\yolov-env\scripts\onnxruntime_test.exe
  Would not remove (might be manually added):
    c:\users\usudl\anaconda3\envs\yolov-env\lib\site-packages\onnxruntime\capi\onnxruntime_providers_cuda.dll
    c:\users\usudl\anaconda3\envs\yolov-env\lib\site-packages\onnxruntime\capi\onnxruntime_providers_tensorrt.dll
Proceed (Y/n)? 



In [ ]:
!pip install tf-keras
!pip install sng4onnx==1.0.1
!pip install onnx_graphsurgeon==0.3.26
!pip install ai-edge-litert==1.2.0
!pip install onnx2tf==1.26.3


In [ ]:
!yolo benchmark model=../trained_models/yolo11n-obb-320.pt data='../datasets/obb_mini_things-3/data.yaml' imgsz=320 device='cuda'

In [ ]:
!pip install openvino-dev[onnx] openvino
!pip install onnx onnxruntime-gpu  # gunakan onnxruntime-gpu jika pakai CUDA
!pip install PyMNN-*-cp310-cp310-linux_x86_64.whl  # sesuai Python & OS kamu

In [7]:
import os
from ultralytics import YOLO

# Daftar model YOLOv11 OBB yang akan dikonversi
models = [
    "yolo11n-obb-320.pt",
    "yolo11n-obb-640.pt",
    "yolo11n-obb-1280.pt",
    "yolo11s-obb-320.pt",
    "yolo11s-obb-640.pt",
    "yolo11s-obb-1280.pt",
    "yolo11m-obb-320.pt",
    "yolo11m-obb-640.pt",
    "yolo11m-obb-1280.pt",
]

# Format target yang ingin diekspor
export_formats = ["onnx", "openvino", "mnn", "ncnn"]  # bisa tambah: 'coreml', 'tflite', 'engine', dll

# Path model dan output
model_dir = "../trained_models"
export_dir = "../exported_models_pc"

os.makedirs(export_dir, exist_ok=True)

for model_name in models:
    model_path = os.path.join(model_dir, model_name)

    if not os.path.exists(model_path):
        print(f"[SKIP] Model tidak ditemukan: {model_path}")
        continue

    print(f"\n🚀 Mengonversi {model_name}...")
    model = YOLO(model_path)

    for fmt in export_formats:
        try:
            print(f"   🔁 Export ke format: {fmt}")
            model.export(format=fmt, imgsz=640, device=0)
        except Exception as e:
            print(f"   ❌ Gagal export ke {fmt}: {e}")



🚀 Mengonversi yolo11n-obb-320.pt...
   🔁 Export ke format: onnx
Ultralytics 8.3.149  Python-3.10.16 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 4070 Ti SUPER, 16376MiB)
YOLO11n-obb summary (fused): 109 layers, 2,661,523 parameters, 0 gradients, 6.6 GFLOPs

PyTorch: starting from '..\trained_models\yolo11n-obb-320.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 45, 8400) (5.4 MB)

ONNX: starting export with onnx 1.17.0 opset 19...
ONNX: slimming with onnxslim 0.1.56...
ONNX: export success  2.3s, saved as '..\trained_models\yolo11n-obb-320.onnx' (10.3 MB)

Export complete (3.1s)
Results saved to C:\Users\usudl\Fitra\obb-sbc-benchmark\trained_models
Predict:         yolo predict task=obb model=..\trained_models\yolo11n-obb-320.onnx imgsz=640  
Validate:        yolo val task=obb model=..\trained_models\yolo11n-obb-320.onnx imgsz=640 data=C:\Users\usudl\Fitra\OBB-YOLO\datasets\obb_mini_things-3/data.yaml  
Visualize:       https://netron.app
   🔁 Export ke format: openvino
U

In [5]:
import os
import json
import csv
import torch
import platform
import psutil
from ultralytics import YOLO

def get_system_info():
    return {
        "device": "cuda" if torch.cuda.is_available() else "cpu",
        "cuda_name": torch.cuda.get_device_name(0) if torch.cuda.is_available() else None,
        "cuda_memory_total_MB": torch.cuda.get_device_properties(0).total_memory // (1024 * 1024) if torch.cuda.is_available() else None,
        "torch_version": torch.__version__,
        "python_version": platform.python_version(),
        "os": platform.system(),
        "os_version": platform.version(),
        "cpu": platform.processor(),
        "cpu_count": psutil.cpu_count(logical=True),
        "ram_total_GB": round(psutil.virtual_memory().total / (1024 ** 3), 2),
    }

models_base = [
    ("yolo11n-obb", 320),
    ("yolo11n-obb", 640),
    ("yolo11n-obb", 1280),
    ("yolo11s-obb", 320),
    ("yolo11s-obb", 640),
    ("yolo11s-obb", 1280),
    ("yolo11m-obb", 320),
    ("yolo11m-obb", 640),
    ("yolo11m-obb", 1280),
]

formats = ["pt", "onnx", "openvino", "mnn", "ncnn"]
model_dir = "../trained_models"
dataset_path = "../datasets/obb_mini_things-3/data.yaml"
save_dir = "validation_outputs"

benchmark_results = []

for fmt in formats:
    for base_name, size in models_base:
        model_name = f"{base_name}-{size}.{fmt}"
        model_path = os.path.join(model_dir, model_name)
        output_name = model_name.replace(f".{fmt}", "")

        if not os.path.exists(model_path):
            print(f"[SKIP] Model tidak ditemukan: {model_path}")
            continue

        print(f"\n🔎 Validating {model_name} at {size}px...")

        entry = {
            "model": model_name,
            "format": fmt,
            "imgsz": size,
            "device": "cuda" if torch.cuda.is_available() else "cpu",
            "mAP50": None,
            "mAP50-95": None,
            "precision": None,
            "recall": None,
            "inference_time_ms": None,
            "fps": None,
            "note": "",
            **get_system_info()
        }

        try:
            if fmt == "pt":
                model = YOLO(model_path)
                results = model.val(
                    data=dataset_path,
                    imgsz=size,
                    batch=1,
                    conf=0.25,
                    iou=0.6,
                    device=0,
                    project=save_dir,
                    name=output_name,
                    save_txt=True,
                    save_conf=True,
                    plots=True,
                    verbose=False
                )

                entry.update({
                    "mAP50": results.box.map50,
                    "mAP50-95": results.box.map,
                    "precision": results.box.mp,
                    "recall": results.box.mr,
                    "inference_time_ms": results.speed['inference'],
                    "fps": 1000 / results.speed['inference'] if results.speed['inference'] else 0
                })

            else:
                entry["note"] = f"Format {fmt.upper()} belum didukung langsung oleh Ultralytics."

        except Exception as e:
            entry["note"] = f"ERROR: {str(e)}"

        benchmark_results.append(entry)

# Simpan hasil ke JSON
os.makedirs(save_dir, exist_ok=True)
json_path = os.path.join(save_dir, "validation_benchmark.json")
with open(json_path, "w") as f_json:
    json.dump(benchmark_results, f_json, indent=2)

# Simpan hasil ke CSV
csv_path = os.path.join(save_dir, "validation_benchmark.csv")
with open(csv_path, "w", newline="") as f_csv:
    writer = csv.DictWriter(f_csv, fieldnames=benchmark_results[0].keys())
    writer.writeheader()
    writer.writerows(benchmark_results)

print(f"\n✅ Semua hasil disimpan ke:\n- JSON: {json_path}\n- CSV : {csv_path}")



🔎 Validating yolo11n-obb-320.pt at 320px...
Ultralytics 8.3.149  Python-3.10.16 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 4070 Ti SUPER, 16376MiB)
YOLO11n-obb summary (fused): 109 layers, 2,661,523 parameters, 0 gradients, 6.6 GFLOPs
val: Fast image access  (ping: 0.00.0 ms, read: 135.720.2 MB/s, size: 16.7 KB)


val: Scanning C:\Users\usudl\Fitra\obb-sbc-benchmark\datasets\obb_mini_things-3\valid\labels.cache... 121 images, 0 bac
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 121/121 [00:


                   all        121        190      0.973      0.991      0.993       0.93
Speed: 0.6ms preprocess, 10.4ms inference, 0.0ms loss, 3.0ms postprocess per image
Results saved to validation_outputs\yolo11n-obb-320

🔎 Validating yolo11n-obb-640.pt at 640px...
Ultralytics 8.3.149  Python-3.10.16 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 4070 Ti SUPER, 16376MiB)
YOLO11n-obb summary (fused): 109 layers, 2,661,523 parameters, 0 gradients, 6.6 GFLOPs
val: Fast image access  (ping: 0.00.0 ms, read: 381.6126.5 MB/s, size: 20.9 KB)


val: Scanning C:\Users\usudl\Fitra\obb-sbc-benchmark\datasets\obb_mini_things-3\valid\labels.cache... 121 images, 0 bac
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 121/121 [00:


                   all        121        190      0.981      0.992      0.994      0.937
Speed: 2.4ms preprocess, 18.0ms inference, 0.0ms loss, 6.3ms postprocess per image
Results saved to validation_outputs\yolo11n-obb-640

🔎 Validating yolo11n-obb-1280.pt at 1280px...
Ultralytics 8.3.149  Python-3.10.16 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 4070 Ti SUPER, 16376MiB)
YOLO11n-obb summary (fused): 109 layers, 2,661,523 parameters, 0 gradients, 6.6 GFLOPs
val: Fast image access  (ping: 0.00.0 ms, read: 309.185.7 MB/s, size: 17.2 KB)


...

                   all        121        190      0.995      0.996      0.993      0.941
Speed: 0.9ms preprocess, 17.0ms inference, 0.0ms loss, 3.7ms postprocess per image
Results saved to validation_outputs\yolo11m-obb-320

🔎 Validating yolo11m-obb-640.pt at 640px...
Ultralytics 8.3.149  Python-3.10.16 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 4070 Ti SUPER, 16376MiB)
YOLO11m-obb summary (fused): 134 layers, 20,909,323 parameters, 0 gradients, 71.5 GFLOPs
val: Fast image access  (ping: 0.00.0 ms, read: 328.9112.6 MB/s, size: 18.4 KB)


val: Scanning C:\Users\usudl\Fitra\obb-sbc-benchmark\datasets\obb_mini_things-3\valid\labels.cache... 121 images, 0 bac
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 121/121 [00:


                   all        121        190          1          1      0.995      0.955
Speed: 0.3ms preprocess, 10.8ms inference, 0.0ms loss, 2.0ms postprocess per image
Results saved to validation_outputs\yolo11m-obb-640

🔎 Validating yolo11m-obb-1280.pt at 1280px...
Ultralytics 8.3.149  Python-3.10.16 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 4070 Ti SUPER, 16376MiB)
YOLO11m-obb summary (fused): 134 layers, 20,909,323 parameters, 0 gradients, 71.5 GFLOPs
val: Fast image access  (ping: 0.10.0 ms, read: 246.7152.6 MB/s, size: 17.0 KB)


val: Scanning C:\Users\usudl\Fitra\obb-sbc-benchmark\datasets\obb_mini_things-3\valid\labels.cache... 121 images, 0 bac
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 121/121 [00:


                   all        121        190      0.988      0.999      0.995      0.939
Speed: 0.6ms preprocess, 18.9ms inference, 0.0ms loss, 3.2ms postprocess per image
Results saved to validation_outputs\yolo11m-obb-1280

🔎 Validating yolo11n-obb-320.onnx at 320px...
[SKIP] Model tidak ditemukan: ../trained_models\yolo11n-obb-640.onnx
[SKIP] Model tidak ditemukan: ../trained_models\yolo11n-obb-1280.onnx
[SKIP] Model tidak ditemukan: ../trained_models\yolo11s-obb-320.onnx
[SKIP] Model tidak ditemukan: ../trained_models\yolo11s-obb-640.onnx
[SKIP] Model tidak ditemukan: ../trained_models\yolo11s-obb-1280.onnx
[SKIP] Model tidak ditemukan: ../trained_models\yolo11m-obb-320.onnx
[SKIP] Model tidak ditemukan: ../trained_models\yolo11m-obb-640.onnx
[SKIP] Model tidak ditemukan: ../trained_models\yolo11m-obb-1280.onnx
[SKIP] Model tidak ditemukan: ../trained_models\yolo11n-obb-320.openvino
[SKIP] Model tidak ditemukan: ../trained_models\yolo11n-obb-640.openvino
[SKIP] Model tidak ditemuka

In [ ]:
import os
import json
import csv
import torch
import platform
import psutil
import re
from ultralytics import YOLO

def get_system_info(device_override=None):
    device = device_override if device_override else ("cuda" if torch.cuda.is_available() else "cpu")
    return {
        "device": device,
        "cuda_name": torch.cuda.get_device_name(0) if device == "cuda" else None,
        "cuda_memory_total_MB": torch.cuda.get_device_properties(0).total_memory // (1024 * 1024) if device == "cuda" else None,
        "torch_version": torch.__version__,
        "python_version": platform.python_version(),
        "os": platform.system(),
        "os_version": platform.version(),
        "cpu": platform.processor(),
        "cpu_count": psutil.cpu_count(logical=True),
        "ram_total_GB": round(psutil.virtual_memory().total / (1024 ** 3), 2),
    }

def extract_resolution(name):
    match = re.search(r'-(\d{3,4})(?:\D|$)', name)
    if match:
        return int(match.group(1))
    return None

# Direktori utama
model_dir = "../trained_models"
dataset_path = "../datasets/obb_mini_things-3/data.yaml"
save_dir = "validation_outputs"
os.makedirs(save_dir, exist_ok=True)

# Deteksi semua file model
model_files = sorted([f for f in os.listdir(model_dir) if f.endswith((".pt", ".onnx", ".torchscript", ".mnn"))])
model_dirs = sorted([f for f in os.listdir(model_dir) if os.path.isdir(os.path.join(model_dir, f)) and "_model" in f])
models = model_files + model_dirs

devices = []
if torch.cuda.is_available():
    devices.append("cuda")
devices.append("cpu")  # selalu jalankan juga versi CPU

benchmark_results = []

for device in devices:
    for model_name in models:
        model_path = os.path.join(model_dir, model_name)
        base_name = model_name.replace("_model", "").replace(".pt", "").replace(".onnx", "").replace(".torchscript", "").replace(".mnn", "")

        # Format dari ekstensi atau sufiks direktori
        if model_name.endswith(".pt"):
            fmt = "pt"
        elif model_name.endswith(".onnx"):
            fmt = "onnx"
        elif model_name.endswith(".mnn"):
            fmt = "mnn"
        elif model_name.endswith(".torchscript"):
            fmt = "torchscript"
        elif "openvino" in model_name:
            fmt = "openvino"
        elif "ncnn" in model_name:
            fmt = "ncnn"
        else:
            fmt = "unknown"

        resolution = extract_resolution(base_name)

        print(f"\n🔎 Validating {model_name} | format={fmt} | size={resolution} | device={device}")

        entry = {
            "model": model_name,
            "format": fmt,
            "imgsz": resolution,
            "device": device,
            "mAP50": None,
            "mAP50-95": None,
            "precision": None,
            "recall": None,
            "inference_time_ms": None,
            "fps": None,
            "note": "",
            **get_system_info(device_override=device)
        }

        try:
            if fmt == "pt":
                model = YOLO(model_path)
                results = model.val(
                    data=dataset_path,
                    imgsz=resolution,
                    batch=1,
                    conf=0.25,
                    iou=0.6,
                    device=0 if device == "cuda" else "cpu",
                    project=save_dir,
                    name=f"{base_name}_{device}",
                    save_txt=True,
                    save_conf=True,
                    plots=False,
                    verbose=False
                )
                entry.update({
                    "mAP50": results.box.map50,
                    "mAP50-95": results.box.map,
                    "precision": results.box.mp,
                    "recall": results.box.mr,
                    "inference_time_ms": results.speed['inference'],
                    "fps": 1000 / results.speed['inference'] if results.speed['inference'] else 0
                })
            else:
                entry["note"] = f"Format {fmt.upper()} belum didukung validasi otomatis."

        except Exception as e:
            entry["note"] = f"[ERROR] {str(e)}"

        benchmark_results.append(entry)

# Simpan hasil ke JSON
json_path = os.path.join(save_dir, "validation_benchmark.json")
with open(json_path, "w") as f_json:
    json.dump(benchmark_results, f_json, indent=2)

# Simpan hasil ke CSV
csv_path = os.path.join(save_dir, "validation_benchmark.csv")
with open(csv_path, "w", newline="") as f_csv:
    writer = csv.DictWriter(f_csv, fieldnames=benchmark_results[0].keys())
    writer.writeheader()
    writer.writerows(benchmark_results)

print(f"\n✅ Semua hasil disimpan ke:\n- JSON: {json_path}\n- CSV : {csv_path}")



🔎 Validating yolo11m-obb-1280.mnn | format=mnn | size=1280 | device=cuda

🔎 Validating yolo11m-obb-1280.onnx | format=onnx | size=1280 | device=cuda

🔎 Validating yolo11m-obb-1280.pt | format=pt | size=1280 | device=cuda
Ultralytics 8.3.149  Python-3.10.16 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 4070 Ti SUPER, 16376MiB)
YOLO11m-obb summary (fused): 134 layers, 20,909,323 parameters, 0 gradients, 71.5 GFLOPs
val: Fast image access  (ping: 0.00.0 ms, read: 455.1129.8 MB/s, size: 18.2 KB)


val: Scanning C:\Users\usudl\Fitra\obb-sbc-benchmark\datasets\obb_mini_things-3\valid\labels.cache... 121 images, 0 bac
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 121/121 [00:

                   all        121        190      0.988      0.999      0.995      0.939


Speed: 0.5ms preprocess, 18.9ms inference, 0.0ms loss, 2.7ms postprocess per image

🔎 Validating yolo11m-obb-1280.torchscript | format=torchscript | size=1280 | device=cuda

🔎 Validating yolo11m-obb-320.mnn | format=mnn | size=320 | device=cuda

🔎 Validating yolo11m-obb-320.onnx | format=onnx | size=320 | device=cuda

🔎 Validating yolo11m-obb-320.pt | format=pt | size=320 | device=cuda
Ultralytics 8.3.149  Python-3.10.16 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 4070 Ti SUPER, 16376MiB)
YOLO11m-obb summary (fused): 134 layers, 20,909,323 parameters, 0 gradients, 71.5 GFLOPs
val: Fast image access  (ping: 0.00.0 ms, read: 251.366.8 MB/s, size: 17.6 KB)


val: Scanning C:\Users\usudl\Fitra\obb-sbc-benchmark\datasets\obb_mini_things-3\valid\labels.cache... 121 images, 0 bac
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 121/121 [00:

...

Speed: 0.5ms preprocess, 8.8ms inference, 0.0ms loss, 2.5ms postprocess per image

🔎 Validating yolo11s-obb-1280.torchscript | format=torchscript | size=1280 | device=cuda

🔎 Validating yolo11s-obb-320.mnn | format=mnn | size=320 | device=cuda

🔎 Validating yolo11s-obb-320.onnx | format=onnx | size=320 | device=cuda

🔎 Validating yolo11s-obb-320.pt | format=pt | size=320 | device=cuda
Ultralytics 8.3.149  Python-3.10.16 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 4070 Ti SUPER, 16376MiB)
YOLO11s-obb summary (fused): 109 layers, 9,714,267 parameters, 0 gradients, 22.4 GFLOPs
val: Fast image access  (ping: 0.00.0 ms, read: 277.378.4 MB/s, size: 18.9 KB)


val: Scanning C:\Users\usudl\Fitra\obb-sbc-benchmark\datasets\obb_mini_things-3\valid\labels.cache... 121 images, 0 bac
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 121/121 [00:

                   all        121        190      0.979      0.999      0.995      0.945


Speed: 0.3ms preprocess, 8.4ms inference, 0.0ms loss, 2.3ms postprocess per image

🔎 Validating yolo11s-obb-320.torchscript | format=torchscript | size=320 | device=cuda

🔎 Validating yolo11s-obb-640.mnn | format=mnn | size=640 | device=cuda

🔎 Validating yolo11s-obb-640.onnx | format=onnx | size=640 | device=cuda

🔎 Validating yolo11s-obb-640.pt | format=pt | size=640 | device=cuda
Ultralytics 8.3.149  Python-3.10.16 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 4070 Ti SUPER, 16376MiB)
YOLO11s-obb summary (fused): 109 layers, 9,714,267 parameters, 0 gradients, 22.4 GFLOPs
val: Fast image access  (ping: 0.00.0 ms, read: 364.4118.3 MB/s, size: 17.4 KB)


val: Scanning C:\Users\usudl\Fitra\obb-sbc-benchmark\datasets\obb_mini_things-3\valid\labels.cache... 121 images, 0 bac
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95):   1%|          | 1/121 [00:00